# About

Convert deeplabv3_mnv2_ade20k_train_2018_12_03 to Core ML

Try to optimize the TF graph before conversion by striping unused nodes. But failed in to convert from TF to Core ML.

# Download model and setup enviroments

In [1]:
!rm -rf deeplabv3_mnv2_ade20k_train_2018_12_03
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz
!tar -zxvf deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz

--2021-08-11 17:42:47--  http://download.tensorflow.org/models/deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.153.128, 2a00:1450:4013:c16::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.153.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26028742 (25M) [application/x-tar]
Saving to: ‘deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz’

deeplabv3_mnv2_ade2 100%[===================>]  24.82M  69.6MB/s    in 0.4s    

2021-08-11 17:42:48 (69.6 MB/s) - ‘deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz’ saved [26028742/26028742]

deeplabv3_mnv2_ade20k_train_2018_12_03/
deeplabv3_mnv2_ade20k_train_2018_12_03/model.ckpt.index
deeplabv3_mnv2_ade20k_train_2018_12_03/frozen_inference_graph.pb
deeplabv3_mnv2_ade20k_train_2018_12_03/model.ckpt.data-00000-of-00001


In [2]:
%tensorflow_version 1.x

!pip install coremltools==4.1

TensorFlow 1.x selected.
     |████████████████████████████████| 3.4 MB 16.4 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=e2cc38e24db77aafb07e1963555b68acc9cc5ee58daf984fdf8a04003e8a0558
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [3]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
1.15.2


4.1


# Inspect the graph

In [4]:
def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

    return graph_def

def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))
  print('Exported to:', filename)


model_name = 'deeplabv3_mnv2_ade20k_train_2018_12_03'
input_file = f'{model_name}/frozen_inference_graph.pb'

with tf.Session(graph=tf.Graph()) as sess:
  graph_def = load_frozen_graph(input_file)
  tf.import_graph_def(graph_def, name="")
  the_graph = tf.compat.v1.get_default_graph()
  export_ops_name(the_graph, f'{model_name}_ops.txt')


Exported to: deeplabv3_mnv2_ade20k_train_2018_12_03_ops.txt


# Optimize the TF graph

Try to optimize the TF graph before conversion by striping unused nodes

In [5]:
input_name = "ImageTensor"
output_name = "SemanticPredictions"

input_w = 513
input_h = 513

tf_frozen_graph_file = 'deeplabv3_mnv2_ade20k_train_2018_12_03/frozen_inference_graph.pb'
optimized_frozen_graph_file = 'optimized_frozen_inference_graph.pb'

In [9]:
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile


def print_node_name(graph_def):
  nodes = [n.name for n in graph_def.node]
  print('nodes', len(nodes), nodes)
  print(nodes[0], '->', nodes[-1])


def optimize_graph(graph_def, 
                   input_node_names, 
                   output_node_names,
                   frozen_model_file):

  print('nodes before strip_unused_lib')
  print_node_name(graph_def)
  
  graph_def = strip_unused_lib.strip_unused(
          input_graph_def = graph_def,
          input_node_names = input_node_names,
          output_node_names = output_node_names,
          placeholder_type_enum = dtypes.float32.as_datatype_enum)
  
  print('nodes after strip_unused_lib')
  print_node_name(graph_def)

  with gfile.GFile(frozen_model_file, "wb") as f:
    f.write(graph_def.SerializeToString())

 
graph_def = load_frozen_graph(input_file)
optimize_graph(graph_def,
              input_node_names=[input_name], 
              output_node_names=[output_name],
              frozen_model_file=optimized_frozen_graph_file)
 

nodes before strip_unused_lib
nodes 1034 ['ImageTensor', 'Shape', 'strided_slice/stack', 'strided_slice/stack_1', 'strided_slice/stack_2', 'strided_slice', 'Squeeze', 'Cast', 'Shape_1', 'strided_slice_1/stack', 'strided_slice_1/stack_1', 'strided_slice_1/stack_2', 'strided_slice_1', 'strided_slice_2/stack', 'strided_slice_2/stack_1', 'strided_slice_2/stack_2', 'strided_slice_2', 'sub/x', 'sub', 'Maximum/y', 'Maximum', 'add', 'sub_1/x', 'sub_1', 'Maximum_1/y', 'Maximum_1', 'add_1', 'Reshape/tensor', 'Reshape/shape', 'Reshape', 'Rank', 'Equal/y', 'Equal', 'Assert/Assert/data_0', 'Assert/Assert/data_1', 'Assert/Assert', 'sub_2', 'Shape_2', 'strided_slice_3/stack', 'strided_slice_3/stack_1', 'strided_slice_3/stack_2', 'strided_slice_3', 'strided_slice_4/stack', 'strided_slice_4/stack_1', 'strided_slice_4/stack_2', 'strided_slice_4', 'GreaterEqual', 'Assert_1/Assert/data_0', 'Assert_1/Assert', 'GreaterEqual_1', 'Assert_2/Assert/data_0', 'Assert_2/Assert', 'sub_3/y', 'sub_3', 'sub_4', 'sub_5

# Convert the TensorFlow model to Core ML.

In [11]:
image_input = ct.ImageType(name=input_name,
                           shape=(1, input_h, input_w, 3),
                           bias=[0,0,0], 
                           scale=1)

coreml_model = ct.convert(
    optimized_frozen_graph_file,
    source='tensorflow',
    inputs=[image_input],
    outputs=[output_name]
)

print(coreml_model)

ValueError: ignored